In [15]:
import json
import whisper

In [6]:
!python --version

Python 3.10.12


In [4]:
%%bash
pip install -U openai-whisper tiktoken
sudo apt update && sudo apt install ffmpeg -y
pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
pip install setuptools-rust

  Using cached tiktoken-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)


Hit:1 http://ap-southeast-2.ec2.archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://ap-southeast-2.ec2.archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://ap-southeast-2.ec2.archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:5 http://dl.google.com/linux/chrome-remote-desktop/deb stable InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://dl.google.com/linux/chrome-remote-desktop/deb stable Release
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Reading package lists...
Building dependency tree...
Reading state information...
24 packages can be upgraded. Run 'apt list --upgradable' to see them.


W: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.




Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libavdevice58 libdc1394-25 libdecor-0-0 libdecor-0-plugin-1-cairo
  libopenal-data libopenal1 libsdl2-2.0-0 libsndio7.0
Suggested packages:
  ffmpeg-doc libportaudio2 sndiod
The following NEW packages will be installed:
  ffmpeg libavdevice58 libdc1394-25 libdecor-0-0 libdecor-0-plugin-1-cairo
  libopenal-data libopenal1 libsdl2-2.0-0 libsndio7.0
0 upgraded, 9 newly installed, 0 to remove and 24 not upgraded.
Need to get 3217 kB of archives.
After this operation, 6006 kB of additional disk space will be used.
Get:1 http://ap-southeast-2.ec2.archive.ubuntu.com/ubuntu jammy/universe amd64 libdc1394-25 amd64 2.2.6-4 [88.8 kB]
Get:2 http://ap-southeast-2.ec2.archive.ubuntu.com/ubuntu jammy/universe amd64 libopenal-data all 1:1.19.1-2build3 [164 kB]
Get:3 http://ap-southeast-2.ec2.archive.ubuntu.com/ubuntu jammy/universe amd64 libsndio7.0 amd64 1.8.1-1.1 [

dpkg-preconfigure: unable to re-open stdin: No such file or directory


Fetched 3217 kB in 0s (11.5 MB/s)
Selecting previously unselected package libdc1394-25:amd64.
(Reading database ... 214036 files and directories currently installed.)
Preparing to unpack .../0-libdc1394-25_2.2.6-4_amd64.deb ...
Unpacking libdc1394-25:amd64 (2.2.6-4) ...
Selecting previously unselected package libopenal-data.
Preparing to unpack .../1-libopenal-data_1%3a1.19.1-2build3_all.deb ...
Unpacking libopenal-data (1:1.19.1-2build3) ...
Selecting previously unselected package libsndio7.0:amd64.
Preparing to unpack .../2-libsndio7.0_1.8.1-1.1_amd64.deb ...
Unpacking libsndio7.0:amd64 (1.8.1-1.1) ...
Selecting previously unselected package libopenal1:amd64.
Preparing to unpack .../3-libopenal1_1%3a1.19.1-2build3_amd64.deb ...
Unpacking libopenal1:amd64 (1:1.19.1-2build3) ...
Selecting previously unselected package libdecor-0-0:amd64.
Preparing to unpack .../4-libdecor-0-0_0.1.0-3build1_amd64.deb ...
Unpacking libdecor-0-0:amd64 (0.1.0-3build1) ...
Selecting previously unselected pa

In [174]:
!nvidia-smi

Fri Jul 21 14:23:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   26C    P0    24W /  70W |  13440MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Preprocess

In [ ]:
%%bash
ffmpeg -i P1.m4a -ar 16000 -ac 1 -c:a pcm_s16le p1.wav

In [ ]:
%%bash
ffmpeg -i p1.wav -ss 00:00:00 -t 00:00:20 p1_20s.wav

## Using the model

### Transcribing

| Param                       | Description                                                                                                                          |
|-----------------------------|--------------------------------------------------------------------------------------------------------------------------------------|
| model                       | The Whisper model instance                                                                                                           |
| audio                       | The path to the audio file to open, or the audio waveform                                                                             |
| verbose                     | Whether to display the text being decoded to the console. If True, displays all the details, If False, displays minimal details. If None, does not display anything |
| temperature                 | Temperature for sampling. It can be a tuple of temperatures, which will be successively used upon failures according to either `compression_ratio_threshold` or `logprob_threshold`. |
| compression_ratio_threshold | If the gzip compression ratio is above this value, treat as failed                                                                     |
| logprob_threshold           | If the average log probability over sampled tokens is below this value, treat as failed                                               |
| no_speech_threshold         | If the no_speech probability is higher than this value AND the average log probability over sampled tokens is below `logprob_threshold`, consider the segment as silent |
| condition_on_previous_text  | if True, the previous output of the model is provided as a prompt for the next window; disabling may make the text inconsistent across windows, but the model becomes less prone to getting stuck in a failure loop, such as repetition looping or timestamps going out of sync. |
| word_timestamps             | Extract word-level timestamps using the cross-attention pattern and dynamic time warping, and include the timestamps for each word in each segment. |
| prepend_punctuations        | If word_timestamps is True, merge these punctuation symbols with the next word                                                        |
| append_punctuations         | If word_timestamps is True, merge these punctuation symbols with the previous word                                                    |
| initial_prompt              | Optional text to provide as a prompt for the first window. This can be used to provide, or "prompt-engineer" a context for transcription, e.g. custom vocabularies or proper nouns to make it more likely to predict those word correctly. |
| decode_options              | Keyword arguments to construct `DecodingOptions` instances                                                                            |

**DecodeOptions**

| Flag                              | Default Value | Description                                                                                               |
|-----------------------------------|---------------|-----------------------------------------------------------------------------------------------------------|
| audio                             | None          | audio file(s) to transcribe                                                                               |
| --model                           | "small"       | name of the Whisper model to use                                                                          |
| --model_dir                       | None          | the path to save model files; uses ~/.cache/whisper by default                                             |
| --device                          | "cuda" or "cpu"| device to use for PyTorch inference                                                                       |
| --output_dir, -o                  | "."           | directory to save the outputs                                                                             |
| --output_format, -f               | "all"         | format of the output file; if not specified, all available formats will be produced                      |
| --verbose                         | True          | whether to print out the progress and debug messages                                                      |
| --task                            | "transcribe" or "translate" | whether to perform X->X speech recognition ('transcribe') or X->English translation ('translate') |
| --language                        | None          | language spoken in the audio, specify None to perform language detection                                   |
| --temperature                     | 0             | temperature to use for sampling                                                                           |
| --best_of                         | 5             | number of candidates when sampling with non-zero temperature                                               |
| --beam_size                       | 5             | number of beams in beam search, only applicable when temperature is zero                                   |
| --patience                        | None          | optional patience value to use in beam decoding, as in https://arxiv.org/abs/2204.05424                   |
| --length_penalty                  | None          | optional token length penalty coefficient (alpha) as in https://arxiv.org/abs/1609.08144                   |
| --suppress_tokens                 | "-1"          | comma-separated list of token ids to suppress during sampling; '-1' will suppress most special characters  |
| --initial_prompt                  | None          | optional text to provide as a prompt for the first window.                                                |
| --condition_on_previous_text      | True          | if True, provide the previous output of the model as a prompt for the next window                         |
| --fp16                            | True          | whether to perform inference in fp16                                                                      |
| --temperature_increment_on_fallback | 0.2           | temperature to increase when falling back when the decoding fails to meet certain thresholds               |
| --compression_ratio_threshold     | 2.4           | if the gzip compression ratio is higher than this value, treat the decoding as failed                     |
| --logprob_threshold               | -1.0          | if the average log probability is lower than this value, treat the decoding as failed                      |
| --no_speech_threshold             | 0.6           | if the probability of the <|nospeech|> token is higher than this value AND the decoding has failed, consider the segment as silence |
| --word_timestamps                 | False         | (experimental) extract word-level timestamps and refine the results based on them                          |
| --prepend_punctuations            | "\"\'“¿([{-" | if word_timestamps is True, merge these punctuation symbols with the next word                           |
| --append_punctuations             | "\"\'.。,，!！?？:：”)]}、" | if word_timestamps is True, merge these punctuation symbols with the previous word                 |
| --threads                         | 0             | number of threads used by torch for CPU inference; supersedes MKL_NUM_THREADS/OMP_NUM_THREADS            |

In [28]:
LANGUAGES = {
    "en": "english",
    "zh": "chinese",
    "de": "german",
    "es": "spanish",
    "ru": "russian",
    "ko": "korean",
    "fr": "french",
    "ja": "japanese",
    "pt": "portuguese",
    "tr": "turkish",
    "pl": "polish",
    "ca": "catalan",
    "nl": "dutch",
    "ar": "arabic",
    "sv": "swedish",
    "it": "italian",
    "id": "indonesian",
    "hi": "hindi",
    "fi": "finnish",
    "vi": "vietnamese",
    "he": "hebrew",
    "uk": "ukrainian",
    "el": "greek",
    "ms": "malay",
    "cs": "czech",
    "ro": "romanian",
    "da": "danish",
    "hu": "hungarian",
    "ta": "tamil",
    "no": "norwegian",
    "th": "thai",
    "ur": "urdu",
    "hr": "croatian",
    "bg": "bulgarian",
    "lt": "lithuanian",
    "la": "latin",
    "mi": "maori",
    "ml": "malayalam",
    "cy": "welsh",
    "sk": "slovak",
    "te": "telugu",
    "fa": "persian",
    "lv": "latvian",
    "bn": "bengali",
    "sr": "serbian",
    "az": "azerbaijani",
    "sl": "slovenian",
    "kn": "kannada",
    "et": "estonian",
    "mk": "macedonian",
    "br": "breton",
    "eu": "basque",
    "is": "icelandic",
    "hy": "armenian",
    "ne": "nepali",
    "mn": "mongolian",
    "bs": "bosnian",
    "kk": "kazakh",
    "sq": "albanian",
    "sw": "swahili",
    "gl": "galician",
    "mr": "marathi",
    "pa": "punjabi",
    "si": "sinhala",
    "km": "khmer",
    "sn": "shona",
    "yo": "yoruba",
    "so": "somali",
    "af": "afrikaans",
    "oc": "occitan",
    "ka": "georgian",
    "be": "belarusian",
    "tg": "tajik",
    "sd": "sindhi",
    "gu": "gujarati",
    "am": "amharic",
    "yi": "yiddish",
    "lo": "lao",
    "uz": "uzbek",
    "fo": "faroese",
    "ht": "haitian creole",
    "ps": "pashto",
    "tk": "turkmen",
    "nn": "nynorsk",
    "mt": "maltese",
    "sa": "sanskrit",
    "lb": "luxembourgish",
    "my": "myanmar",
    "bo": "tibetan",
    "tl": "tagalog",
    "mg": "malagasy",
    "as": "assamese",
    "tt": "tatar",
    "haw": "hawaiian",
    "ln": "lingala",
    "ha": "hausa",
    "ba": "bashkir",
    "jw": "javanese",
    "su": "sundanese",
}

In [19]:
MODELS = [
    'tiny.en', 'tiny', 
    'base.en', 'base', 
    'small.en', 'small', 
    'medium.en', 'medium', 
    'large-v1', 'large-v2', 'large'
]

**Use `initial_prompt` to guide the model to use English**

In [25]:
model = whisper.load_model("medium")

In [ ]:
%%time
result = model.transcribe("p1.wav", initial_prompt="This is an interview in English.")

**Use `language` param**

In [43]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 110MiB/s]


In [44]:
%%time

# tiny model 
# CPU times: user 1min 20s, sys: 1.83 s, total: 1min 22s
# Wall time: 1min 20s

# base model
# CPU times: user 1min 20s, sys: 1.56 s, total: 1min 21s
# Wall time: 1min 19s

# medium model
# quên đo

# large model
# CPU times: user 7min 10s, sys: 17.3 s, total: 7min 27s
# Wall time: 7min 25s

options = {
    "language": LANGUAGES["en"]
}
result = model.transcribe(
    "p1.wav", 
    verbose=True, 
    word_timestamps=True, 
    **options
)

[00:02.600 --> 00:07.580]  Okay, do you know of any male APNs in the workforce?
[00:08.680 --> 00:08.940]  Yes.
[00:09.600 --> 00:10.900]  I know of...
[00:10.900 --> 00:13.540]  I mean...
[00:13.540 --> 00:15.600]  One...
[00:15.600 --> 00:16.980]  Two...
[00:16.980 --> 00:17.620]  Two!
[00:18.120 --> 00:18.420]  Yes.
[00:19.020 --> 00:21.200]  So sounds like very few, right?
[00:22.060 --> 00:22.060]  Yes.
[00:22.800 --> 00:26.480]  So do you think that there are very few APNs in the workforce?
[00:27.400 --> 00:29.500]  Yes. There are very few APNs in the workforce.
[00:31.060 --> 00:31.940]  Why do you...
[00:31.940 --> 00:32.760]  Sorry, what's your question?
[00:33.360 --> 00:35.540]  Do you know how many male APNs in the workforce?
[00:36.140 --> 00:40.320]  Do you think that there are very few APNs in the workforce?
[00:41.120 --> 00:43.700]  Yes, there are very few APNs in the workforce.
[00:44.120 --> 00:46.600]  It's so, then why? Why do you think so?
[00:47.460 --> 00:51.30

In [45]:
with open("p1_base.json", "w") as file:
    # Write content to the file
    file.write(json.dumps(result, indent=4))

### Visualise word confidence

In [143]:
from IPython.display import display, HTML

In [165]:
import math
import datetime

# AI generated
def color_hex(number):
    if number < 0 or number > 1:
        return "Invalid number"
    r = int((1 - math.log(number + 1)) * 255)
    g = int((1 - math.log(1 - number + 1)) * 255)
    b = 0
    return f"#{r:02x}{g:02x}{b:02x}"

# AI generated
def t(seconds):
    time_delta = datetime.timedelta(seconds=seconds)
    hours = time_delta.seconds // 3600
    minutes = (time_delta.seconds // 60) % 60
    seconds = time_delta.seconds % 60

    time_text = ""
    if hours > 0:
        time_text += f"{hours}h"
    if minutes > 0 or hours > 0:
        time_text += f"{minutes}m"
    time_text += f"{seconds}s"
    
    return time_text

In [162]:
words = [word for segment in result["segments"] for word in segment["words"]]

In [171]:
to_html = lambda word: f'''
    <span 
        title="probability: {word["probability"]}" 
        style="color: {color_hex(word["probability"])}"
        data-text=" at {t(word["start"])}"
    >
        {word["word"]}
    </span>
'''

html = " ".join([
    to_html(word)
    for word in words
])

In [172]:
css = """
<style>
span:hover::after {
  font-size: 10px;
  color: gray;
  content: attr(data-text);
}
</style>

"""

display(HTML(css + html))

### Model benchmarking

In [ ]:
 ¯\_(ツ)_/¯

In [ ]:
 ¯\_(ツ)_/¯

In [ ]:
 ¯\_(ツ)_/¯

### Diarization (identify speaker)?

https://github.com/openai/whisper/discussions/117#discussioncomment-3727051

In [ ]:
 ¯\_(ツ)_/¯

In [ ]:
 ¯\_(ツ)_/¯

### 70x performance boost?

https://github.com/sanchit-gandhi/whisper-jax

In [ ]:
 ¯\_(ツ)_/¯

### Realtime transcription?

https://github.com/davabase/whisper_real_time

In [ ]:
 ¯\_(ツ)_/¯

### Finetuning? Vietnamese?

In [ ]:
 ¯\_(ツ)_/¯

## Use LLAMA2 for querying data

In [ ]:
# %%bash
# pip install langchain chromadb openai tiktoken

In [38]:
# !pip install sentence_transformers

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain import HuggingFaceHub
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from chromadb.errors import InvalidDimensionException
import json

# Load and process the text
loader = TextLoader('./p1_whispercppb9e5770b6ce7086e25c131b33f1c9bfc1b0b45db823137f9a0c1d32c46089253.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
try:
    db = Chroma.from_documents(texts, embedding_function)
except InvalidDimensionException:
    Chroma().delete_collection()
    db = Chroma.from_documents(texts, embedding_function)

docs = db.similarity_search("Summarize the conversation!", k=5)
print(docs)

/home/ubuntu//envs/hello1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(page_content="[00:54:22.000 --> 00:54:26.000]   I think I covered most of it.\n[00:54:26.000 --> 00:54:29.000]   Yes, I think so too.\n[00:54:29.000 --> 00:54:30.000]   Yeah.\n[00:54:30.000 --> 00:54:31.000]   Okay, thank you.\n[00:54:31.000 --> 00:54:32.000]   I'll take the six.\n[00:54:32.000 --> 00:54:34.000]   Anything else to say?\n[00:54:34.000 --> 00:54:36.000]   No, I'll take the six.\n[00:54:36.000 --> 00:54:38.000]   Let me stop the recording.", metadata={'source': './p1_whispercppb9e5770b6ce7086e25c131b33f1c9bfc1b0b45db823137f9a0c1d32c46089253.txt'}), Document(page_content='[00:47:47.000 --> 00:48:01.000]   I think that my female colleagues were texting me and telling me that, "Hey, actually we miss you at work and you make the workplace a bit more interesting and less boring."\n[00:48:01.000 --> 00:48:12.000]   So I think that we have a good amount of acceptance wherever we go, especially to the younger colleagues.\n[00:48:12.000 --> 00:48:22.000]   Because I thin

In [2]:
!which python

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/ubuntu//envs/hello1/bin/python


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"
model_basename = "meta-llama/Llama-2-7b-chat-hf"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
double_quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_use_double_quant=True,
)

model_double_quant = AutoModelForCausalLM.from_pretrained(model_basename, quantization_config=double_quant_config)


Loading checkpoint shards: 100%|███████████████████████████████████| 2/2 [01:31<00:00, 45.52s/it]


In [4]:
# !pip install pydantic langchain

In [12]:
"""Wrapper around HuggingFace Pipeline APIs."""
import importlib.util
import logging
import torch
from typing import Any, List, Mapping, Optional

from pydantic import Extra

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens

# DEFAULT_MODEL_ID = "gpt2"
# DEFAULT_TASK = "text-generation"
# VALID_TASKS = ("text2text-generation", "text-generation")

# logger = logging.getLogger(_name_)


class CustomStableLM(LLM):
    """Wrapper around HuggingFace Pipeline API.

    To use, you should have the ``transformers`` python package installed.

    Only supports text-generation and text2text-generation for now.

    Example using from_model_id:
        .. code-block:: python
B
            from langchain.llms import HuggingFacePipeline
            hf = HuggingFacePipeline.from_model_id(
                model_id="gpt2", task="text-generation"
            )
    Example passing pipeline in directly:
        .. code-block:: python

            from langchain.llms import HuggingFacePipeline
            from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

            model_id = "gpt2"
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModelForCausalLM.from_pretrained(model_id)
            pipe = pipeline(
                "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
            )
            hf = HuggingFacePipeline(pipeline=pipe)
    """
    tokenizer: Any  
    model: Any  #: :meta private:
    model_id: str = "text-generation"

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    
    @classmethod
    def from_model_id(
        cls,
        model: Any,
        tokenizer: Any
    ) -> LLM:
        """Construct the pipeline object from model_id and task."""
        return cls(
            model = model,
            tokenizer = tokenizer,
            model_id= "LLAMA_2",
            #model_kwargs=_model_kwargs,
            #**kwargs,
        )

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            **{"model_id": self.model_id},
            #**{"model_kwargs": self.model_kwargs},
        }
    
    def generate_text(self, prompt:str)-> str:
     prompt = f"<|USER|>{prompt}<|ASSISTANT|>"
     inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
     with torch.inference_mode():
        tokens = self.model.generate(
            **inputs,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            max_length=3000
            # stopping_criteria=StoppingCriteriaList([StopOnTokens()])
        )
        completion_tokens = tokens[0][inputs['input_ids'].size(1):]
        completion = self.tokenizer.decode(completion_tokens, skip_special_tokens=True)
        return completion

    @property
    def _llm_type(self) -> str:
        return "LLAMA_2"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        text= self.generate_text(prompt)
        if stop is not None:
            # This is a bit hacky, but I can't figure out a better way to enforce
            # stop tokens when making calls to huggingface_hub.
            text = enforce_stop_tokens(text, stop)
        return text

In [13]:
model_double_quant.device

device(type='cuda', index=0)

In [14]:
llm = CustomStableLM.from_model_id(model = model_double_quant, tokenizer = tokenizer)

In [15]:
llm.generate_text("What is the meaning of life")

"The meaning of life is a complex and philosophical question that has been debated throughout human history. everybody has their own answer to this question, and there is no one definitive answer. However, here are some possible perspectives on the meaning of life:\n1. Religious or spiritual beliefs: Many people believe that the meaning of life is to fulfill a divine or spiritual purpose. According to this view, life has a higher purpose or meaning that is connected to a person's beliefs or practices.\n2. Personal fulfillment: Some people believe that the meaning of life is to find personal fulfillment and happiness. According to this view, life is about pursuing one's passions and interests, and finding joy and satisfaction in life's experiences.\n3. Social connections: Others believe that the meaning of life is found in the relationships and connections we make with others. According to this view, life is about building meaningful relationships with family, friends, and community, an

In [19]:
%%time

def transcript_qa(prompt):
  
    # query it
    docs = db.similarity_search(prompt, k=5)
    res = ' '.join([str(elem.page_content) for elem in docs])
    completion = llm.generate_text(f"""
    You are a safe and helpful assistant. Now act as a audio-text transcription analyst. Generate the output that  a audio-text transcription analyst may give.
    You a given the following audio transcription. Only answer the question that could only be found within the audio transcription
    <|TRANSCRIPTION|>{res}<|TRANSCRIPTION|>
    <|QUESTION|>{prompt}<|QUESTION|>
    """)
    print("docs"+res)
    print("Prompt: " + prompt)
    print("Completion: "+ completion + "\n")

transcript_qa("Summarize the conversation!")

transcript_qa("How many speaker are there in this conversation? and how do you know it?")


docs[00:54:22.000 --> 00:54:26.000]   I think I covered most of it.
[00:54:26.000 --> 00:54:29.000]   Yes, I think so too.
[00:54:29.000 --> 00:54:30.000]   Yeah.
[00:54:30.000 --> 00:54:31.000]   Okay, thank you.
[00:54:31.000 --> 00:54:32.000]   I'll take the six.
[00:54:32.000 --> 00:54:34.000]   Anything else to say?
[00:54:34.000 --> 00:54:36.000]   No, I'll take the six.
[00:54:36.000 --> 00:54:38.000]   Let me stop the recording. [00:47:47.000 --> 00:48:01.000]   I think that my female colleagues were texting me and telling me that, "Hey, actually we miss you at work and you make the workplace a bit more interesting and less boring."
[00:48:01.000 --> 00:48:12.000]   So I think that we have a good amount of acceptance wherever we go, especially to the younger colleagues.
[00:48:12.000 --> 00:48:22.000]   Because I think the older colleagues have a very strong mindset that, "Males shouldn't be doing this job and why are they doing this job?"
[00:48:22.000 --> 00:48:36.000]   So t

In [20]:
transcript_qa("Are there any conversation regarding the APN?. If yes answer why are they mentioned that")


docs[00:00:00.000 --> 00:00:08.000]   Okay, do you know of any male APNs in the workforce?
[00:00:08.000 --> 00:00:17.000]   Yes, I know of one, two.
[00:00:17.000 --> 00:00:18.000]   Two?
[00:00:18.000 --> 00:00:19.000]   Yes.
[00:00:19.000 --> 00:00:22.000]   So it sounds like very few right?
[00:00:22.000 --> 00:00:23.000]   Yes.
[00:00:23.000 --> 00:00:27.000]   So do you think that there are very few APNs in the workforce?
[00:00:27.000 --> 00:00:31.000]   Yes, there are very few APNs in the workforce.
[00:00:31.000 --> 00:00:32.000]   Why do?
[00:00:32.000 --> 00:00:33.000]   Sorry, what's your question?
[00:00:33.000 --> 00:00:36.000]   Do you know how many male APNs in the workforce or...
[00:00:36.000 --> 00:00:41.000]   Do you think that there are very few APNs in the workforce? [00:32:23.000 --> 00:32:43.000]   So then it goes back to my question. Why is there still so few males in the APN workforce? Is it they choose not to or are they not being identified? Are there existi

In [21]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Aug  6 00:32:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/